In [ ]:
import time
import os
from datetime import datetime, timedelta
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

In [ ]:
#This cell sets up all necessary functions 

def GettingLastYearDate(date):
    # Convert the input date to a datetime object
    input_date = datetime.strptime(date, "%m/%d/%Y")

    # Calculate the date of the same day in the previous year
    last_year_date = input_date - timedelta(days=365)

    # Format the date as "mm/dd/yyyy"
    formatted_date = last_year_date.strftime("%m/%d/%Y")

    return formatted_date

def download_files(mkt_airline):
    source_folder = r'C:\Users\vishe\Downloads' #(PATH EDIT) CHANGE TO YOUR OWN DOWNLOAD FOLDER
    destination_folder = r'C:\Users\vishe\Documents' #(PATH EDIT) CHANGE TO HERE JUPYTER/PYTHON IS INSTALLED

    files = os.listdir(source_folder)
    sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(source_folder, x)), reverse=True)
    recent_files = sorted_files[:3]

    for i, file_name in enumerate(recent_files):
        source_path = os.path.join(source_folder, file_name)

        # Convert the file to CSV
        if file_name.lower().endswith('.xlsx'):
            if i == 0:
                new_file_name = f'{mkt_airline}LYData.csv'
            elif i == 1:
                new_file_name = f'{mkt_airline}LWPullData.csv'
            elif i == 2:
                new_file_name = f'{mkt_airline}TWPullData.csv'

            csv_file_path = os.path.join(destination_folder, new_file_name)
            data_frame = pd.read_excel(source_path)
            data_frame.to_csv(csv_file_path, index=False)

    print(f"{mkt_airline} Files converted and renamed successfully.")
    

import pandas as pd


def convert_date_format(df):
    df['Travel Date'] = pd.to_datetime(df['Travel Date'])
    df['Travel Date'] = df['Travel Date'].dt.strftime('%b %d, %Y')
    return df



def getCSVs(mkt_airline):
    data_paths = []
    for airline in mkt_airline:
        OGTWData = airline + "TWPullData.csv"
        OGLWData = airline + "LWPullData.csv"
        OGLYData = airline + "LYData.csv"
        if airline in ['XP', 'MX']:
            data_paths.append((OGTWData, OGLWData, OGLYData))
        else:
            OGTwenty19Data = airline + "2019PullData.csv"
            data_paths.append((OGTWData, OGLWData, OGLYData, OGTwenty19Data))
    return data_paths

def cleanCSVs(file_paths):
    cleaned_data = []
    for paths in file_paths:
        TWData = pd.read_csv(paths[0], skiprows=5)
        LWData = pd.read_csv(paths[1], skiprows=5)
        LastYearData = pd.read_csv(paths[2], skiprows=5)
        TWData.dropna(inplace=True)
        LWData.dropna(inplace=True)
        LastYearData.dropna(inplace=True)
        TWData = convert_date_format(TWData)
        LWData = convert_date_format(LWData)
        LYData = convert_date_format(LastYearData)
        if len(paths) == 4:
            Twenty19Data = pd.read_csv(paths[3], skiprows=5)
            Twenty19Data.dropna(inplace=True)
            Twenty19Data = convert_date_format(Twenty19Data)
            cleaned_data.append((TWData, LWData, LYData, Twenty19Data))
        else:
            cleaned_data.append((TWData, LWData, LYData))
    return cleaned_data



def Clean1CSV(file):
    Data=pd.read_csv(file, skiprows=5)
    Data.dropna(inplace=True)
    Data=convert_date_format(Data)
    return Data

## For All Airlines

In [ ]:
#Order of Airline (DO NOT EDIT)
mkt_airline = ['NK','SY','XP','MX','G4','F9','B6','AS','AA','UA','DL','WN']

current_year_date=datetime.now().date().strftime("%m/%d/%Y")
last_year_date=GettingLastYearDate(datetime.now().date().strftime("%m/%d/%Y"))

driver = webdriver.Chrome()

#Diio Credentials
driver.get('https://mi.diio.net/mi/topframe.jsp')
#username="##################""     ####CREDENTIALS REDACTED, CONTACT FOR CREDENTIALS

#password="##################"     ####CREDENTIALS REDACTED, CONTACT FOR CREDENTIALS
username_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, 'username'))
)

# Enter the username
username_input.send_keys(username)

# Wait for the password input field to be visible
password_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, 'password'))
)

# Enter the password
password_input.send_keys(password)

# Wait for the button to be clickable
Continue_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/main/section/div/div/div/form/div[3]/button'))
)

# Click the button
Continue_button.click()

#Finds User tab on Diio and then clicks
UserTab = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/ul/li[1]/a'))
)


UserTab.click()

#Find Saved Reports option on Diio and clicks
SavedReports=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/ul/li[1]/table/tbody/tr/td[2]/ul/li[4]/a'))
)

SavedReports.click()

#Finds arrow to saved report and clicks
ACRArrow=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[5]/div/div/div[3]/form/div[1]/div/div[4]/table/tbody/tr/td[8]/a[1]'))
)

ACRArrow.click()

#Finds text box to enter Airline Code
MarketAirlineSelection=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td/div[1]/span'))
)

#Feed text box Airline Code
MarketAirlineSelection.send_keys(mkt_airline[0])
MarketAirlineSelection.click()

#Finds the start date text box, clears preset date, inserts current date 
UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
)

UpperDateRangeDropDown.clear()
time.sleep(3)
UpperDateRangeDropDown.send_keys(current_year_date)
time.sleep(3)



#Finds Run Report and click
RunReport=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
)

#Runs 'TW' Data
RunReport.click() 

#Necessary Delay to let files download on Diio
time.sleep(5)

#Finds Use Data Loaded drop down and selects 'Last Week'
LastWeekDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td[1]/div/div/table/tbody/tr[2]/td[3]/table/tbody/tr[7]/td/select/option[2]'))
)

LastWeekDropDown.click()
time.sleep(7)

#Runs 'LWData' Report
RunReport.click()
time.sleep(7)

#Finds Use Data Loaded drop down and selects 'This Week'
ThisWeekDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td[1]/div/div/table/tbody/tr[2]/td[3]/table/tbody/tr[7]/td/select/option[1]'))
)
ThisWeekDropDown.click()
time.sleep(4)

#Finds start date text box and enters in last year's date from today
UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
)

UpperDateRangeDropDown.clear()
time.sleep(4)
UpperDateRangeDropDown.send_keys(last_year_date)
time.sleep(4)

#Finds end date text box and replaces last character from text box with last year's date from end of pull period 
LowerDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[3]/input'))
)

LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
time.sleep(3)
LowerDateRangeDropDown.send_keys('3') #<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
time.sleep(3)

#Finds Done button on calendar and clicks
ExitCalendar=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
)

ExitCalendar.click()
time.sleep(1)

#Runs "LYData" Report
RunReport.click()
time.sleep(16)  #<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 

#Takes the last 3 downloads, renames to NKLYData, NKLWData, NKTWData, and saves as CSVs in your folder 
CsvCheck=download_files(mkt_airline[0])

# # #Repeat process above for 'SY' through 'F9' in mkt_airline list at top of cell
for i in range(1,6): #<---- (SECT EDIT) ULCC SECTION Comment Out/Change Loop
    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    time.sleep(2.5)
    MarketAirlineSelection.send_keys(mkt_airline[i])
    time.sleep(2.5)
    UpperDateRangeDropDown.clear()
    time.sleep(2.5)
    UpperDateRangeDropDown.send_keys(current_year_date)
    time.sleep(2.5)
    LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
    time.sleep(2.5)
    #Resetting End Date to match This Week's Data Pull
    LowerDateRangeDropDown.send_keys('4')#<----(KEY EDIT) Change this to '5' when pulling into 2025. Expected date of Change: 04/01/2024
    time.sleep(2.5)


    ExitCalendar=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
    )
    time.sleep(2.5)
    ExitCalendar.click()
    time.sleep(2.5)
    
    RunReport=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
    )
    RunReport.click()

    time.sleep(2.5)
    LastWeekDropDown.click()
    time.sleep(4)
    RunReport.click()
    time.sleep(2.5)
    ThisWeekDropDown.click()
    time.sleep(2.5)
    UpperDateRangeDropDown.clear()
    time.sleep(2.5)
    UpperDateRangeDropDown.send_keys(last_year_date)
    time.sleep(2.5)
    LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
    time.sleep(2.5)
    LowerDateRangeDropDown.send_keys('3')#<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
    time.sleep(2.5)
    ExitCalendar=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
    )

    ExitCalendar.click()
    time.sleep(2.5)

    RunReport.click()
    time.sleep(20)#<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 
    download_files(mkt_airline[i])

Repeat process above for 'B6' and 'AS' in mkt_airline list at top of cell
for i in range(6,8):#<---- (SECT EDIT) MED SECTION Comment Out/Change Loop


    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    time.sleep(3)
    MarketAirlineSelection.send_keys(mkt_airline[i])
    time.sleep(3)
    UpperDateRangeDropDown.clear()
    time.sleep(3)
    UpperDateRangeDropDown.send_keys(current_year_date)
    time.sleep(3)
    LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
    time.sleep(3)
    LowerDateRangeDropDown.send_keys('4')#<----(KEY EDIT) Change this to '5' when pulling into 2025. Expected date of Change: 04/01/2024
    time.sleep(5)


    ExitCalendar=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
    )
    time.sleep(5)
    ExitCalendar.click()
    time.sleep(5)
    RunReport=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
    )
    RunReport.click()
    time.sleep(5)
    LastWeekDropDown.click()
    if mkt_airline[i]=='AS':
        time.sleep(6)
        RunReport=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
    )
        time.sleep(6)
        RunReport.click()
        time.sleep(6.5)
        ThisWeekDropDown.click()
        time.sleep(6.5)
        UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
)
        UpperDateRangeDropDown.clear()
        time.sleep(7)
        UpperDateRangeDropDown.send_keys(last_year_date)
        time.sleep(7)
        LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
        time.sleep(7)
        LowerDateRangeDropDown.send_keys('3')#<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
        time.sleep(7)
        ExitCalendar=WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
        )

        ExitCalendar.click()
        time.sleep(6)

        RunReport.click()
        time.sleep(50)#<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 
        download_files(mkt_airline[i])
    else:
        time.sleep(5)
        RunReport.click()
        time.sleep(5)
        ThisWeekDropDown.click()
        time.sleep(5)
        UpperDateRangeDropDown.clear()
        time.sleep(5)
        UpperDateRangeDropDown.send_keys(last_year_date)
        time.sleep(5)
        LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
        time.sleep(5)
        LowerDateRangeDropDown.send_keys('3')#<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
        time.sleep(5)
        ExitCalendar=WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
        )

        ExitCalendar.click()
        time.sleep(5)

        RunReport.click()
        time.sleep(40)#<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 
        download_files(mkt_airline[i])

#Repeat process above for 'AA' through 'WN' in mkt_airline list at top of cell
for i in range(8,12):#<---- (SECT EDIT) LARGE SECTION Comment Out/Change Loop
    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    MarketAirlineSelection.send_keys(Keys.BACKSPACE)
    time.sleep(3)
    MarketAirlineSelection.send_keys(mkt_airline[i])
    time.sleep(3)
    UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
     EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
 )
    UpperDateRangeDropDown.clear()
    time.sleep(3)
    UpperDateRangeDropDown.send_keys(current_year_date)
    time.sleep(3)
    LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
    time.sleep(3)
    LowerDateRangeDropDown.send_keys('4')#<----(KEY EDIT) Change this to '5' when pulling into 2025. Expected date of Change: 04/01/2024
    time.sleep(5)


    ExitCalendar=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
    )
    time.sleep(5)
    ExitCalendar.click()
    time.sleep(5)
    RunReport.click()

    RunReport=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
    )
    time.sleep(30)
    LastWeekDropDown.click()
    time.sleep(30)
    RunReport.click()
    time.sleep(30)
    ThisWeekDropDown.click()
    time.sleep(15)
    UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
     EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
 )
    UpperDateRangeDropDown.clear()
    time.sleep(15)
    UpperDateRangeDropDown.send_keys(last_year_date)
    time.sleep(9)
    LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
    time.sleep(7)
    LowerDateRangeDropDown.send_keys('3')#<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
    time.sleep(7)
    ExitCalendar=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
    )

    ExitCalendar.click()
    time.sleep(3)

    RunReport.click()
    time.sleep(60)#<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 
    download_files(mkt_airline[i])

## For One Airline Only

In [ ]:
mkt_airline = ['AA'] #<----(MKT EDIT) Insert whatever Airline you want into here 
#Will Be slower to accomodate for large airlines

current_year_date=datetime.now().date().strftime("%m/%d/%Y")
last_year_date=GettingLastYearDate(datetime.now().date().strftime("%m/%d/%Y"))

driver = webdriver.Chrome()

driver.get('https://mi.diio.net/mi/topframe.jsp')
#username="##################""     ####CREDENTIALS REDACTED, CONTACT FOR CREDENTIALS

#password="##################"     ####CREDENTIALS REDACTED, CONTACT FOR CREDENTIALS
username_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, 'username'))
)

# Enter the username
username_input.send_keys(username)

# Wait for the password input field to be visible
password_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, 'password'))
)

# Enter the password
password_input.send_keys(password)

# Wait for the button to be clickable
Continue_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/main/section/div/div/div/form/div[3]/button'))
)

# Click the button
Continue_button.click()

UserTab = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/ul/li[1]/a'))
)

UserTab.click()

SavedReports=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/ul/li[1]/table/tbody/tr/td[2]/ul/li[4]/a'))
)

SavedReports.click()

ACRArrow=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[5]/div/div/div[3]/form/div[1]/div/div[4]/table/tbody/tr/td[8]/a[1]'))
)

ACRArrow.click()

MarketAirlineSelection=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td/div[1]/span'))
)


MarketAirlineSelection.send_keys(mkt_airline[0])
MarketAirlineSelection.click()
UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
)

UpperDateRangeDropDown.clear()
time.sleep(3)
UpperDateRangeDropDown.send_keys(current_year_date)
time.sleep(3)




RunReport=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[2]/div[2]/a'))
)

RunReport.click() 

time.sleep(30)

LastWeekDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td[1]/div/div/table/tbody/tr[2]/td[3]/table/tbody/tr[7]/td/select/option[2]'))
)

LastWeekDropDown.click()
time.sleep(30)

RunReport.click()
time.sleep(30)

ThisWeekDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td[1]/div/div/table/tbody/tr[2]/td[3]/table/tbody/tr[7]/td/select/option[1]'))
)
ThisWeekDropDown.click()
time.sleep(15)

UpperDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[2]/input'))
)

UpperDateRangeDropDown.clear()
time.sleep(15)
UpperDateRangeDropDown.send_keys(last_year_date)
time.sleep(9)

LowerDateRangeDropDown=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[5]/div[1]/form/div[1]/table[2]/tbody/tr/td/div/div[1]/table/tbody/tr[2]/td[3]/table/tbody/tr[5]/td/div/div[3]/input'))
)

LowerDateRangeDropDown.send_keys(Keys.BACKSPACE)
time.sleep(7)
LowerDateRangeDropDown.send_keys('3')#<----(KEY EDIT) Change this to '4' when pulling into 2025. Expected date of Change: 04/01/2024
time.sleep(7)


ExitCalendar=WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[2]/button[2]'))
)

ExitCalendar.click()
time.sleep(3)

RunReport.click()
time.sleep(65)#<---- (TIME EDIT) Change value of time.sleep() if download_files is not working right 

CsvCheck=download_files(mkt_airline[0])